## The goal of this project is to train a classification model that can accurately predict a samples Iron Deficiency Chlorosis (IDC) rating, given the percentages of green, yellow, and brown that exists within each height-wise segment of the sample.

## Model Highlights

### Used Under- and Over-sampling techniques to train a classifier on an unbalanced dataset. 

### Each sample (data point) consists of 3 voxelated 3D point clouds that represent height-wise slices of a parent point voxelated point cloud.

### Because the 3 slices are ordered (bottom to top), a reorder scheme using random selection of permutations was used to arrange the 3 vectors for each datapoint. This scheme would ensure the model is not learning to classify features based on the order of the slices but rather on the color features of the corresponding slices...REMOVING SPATIAL DEPENDENCY

## 3 models (M0, M1, & M2) are trained. Training data consists of color percentages as features (vector [%green %yellow %brown]) and IDC ratings as labels (IDC Bins for Model M0, see below).

**Although Model MB is trained, it is not used in the final hierarchical model.**

### The training data for Model MB is left as-is. Features are percentages of green, yellow, and brown for each sample, and labels are the respective IDC rating for each sample.


## <u>Model M0 Training Data</u>
### The training data for Model M0 is first binned. 
* Samples with IDC ratings 1 & 2 belong to Bin 1. 
* Samples with IDC rating 3 belongs to Bin 2.
* Samples with IDC ratings 4 & 5 belong to Bin 3.

Therefore, the features are the color percentages, and the labels are the bin numbers.

## <u>Model M1 Training Data</u>
### The training data for Model M1 consists of only samples where the IDC rating is 1 or 2.
The features are the color percentages, and the labels are ratings 1 or 2.


## <u>Model M2 Training Data</u>
### The training data for Model M2 consists of only samples where the IDC rating is 4 or 5.
The features are the color percentages, and the labels are ratings 4 or 5.


## The final hierarchical model uses the following logic:

1. Use Model M0 to predict the bin (1, 2, or 3) the sample belongs to.
2. If the predicted bin is 1, use Model M1 to predict an IDC rating of 1 or 2.
3. Else if Model M0 predicts the bin to be 2, return an IDC Rating of 3.
4. Else if Model M0 predicts the bin to be 3, use Model M2 to predict and IDC Rating of 4 or 5.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import imblearn
print(imblearn.__version__)
#import models and performance metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import ComplementNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn import svm
from sklearn import svm as svm_linear

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split as tts
import sklearn
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from matplotlib import pyplot
import pandas as pd

print(imblearn.__version__)
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.over_sampling import SVMSMOTE
from imblearn.over_sampling import ADASYN
from imblearn.over_sampling import KMeansSMOTE


import dataframe_image as dfi

from itertools import permutations
import random

0.11.0
0.11.0


In [2]:
import os

In [3]:
# read input data
df = pd.read_csv('3slice_colerpercentages_output.csv',index_col = False)

In [4]:
df.rename(columns = {'percgreen':'%g','percyellow':'%y','percbrown':'%b','idc_score':'rating'},inplace=True)

In [5]:
df['rating'] = df['rating'].astype(int)

In [6]:
#This function will create the features array for the height-wise datasets 
#(e.g. For 3-Slice data, every three rows of color percentage (%b, %g, %y) data should represent a single datapoint
# and one target label should be applied to that array of 3 vectors). This ensures that the three slices are treated as a single
#datapoint

def slice_feature_arrays(arg1,arg2):
    
    '''
    arg1: data/array containing height-wise feature data
    arg2: number of rows/vectors in each datapoint (number of slices)   
    '''

    #select first 3 rows (vectors) for desired fields and append to array
    X_features = np.array([[np.array(arg1[['%b','%y','%g']][0*arg2:((0*arg2)+arg2)])]])
    
    #reshape features array
    X_features = X_features.reshape(X_features.shape[0],arg2,3)
    
    #calculate permutation of 3 vectors (%b, %g, %y) within each array
    feature_perm_lst_0 = list(permutations(X_features[0]))
    
    #use random.choice to select random permuation from list
    X_features = np.array([random.choice(feature_perm_lst_0)])

  


    # iterate through dataframe and append desired rows (vectors) for the desired fields to current array
    # the desired number of rows/vectors represents a single datapoint
    # range begins at '1' since above array already contains first datapoint

    for i in range(1,(int(arg1.shape[0]/arg2))):

        
        feature_ord = np.array([[np.array(arg1[['%b','%y','%g']][i*arg2:((i*arg2)+arg2)])]])
        
        #calculate permutation of 3 vectors (%b, %g, %y) within each array
        feature_perm_lst = list(permutations(feature_ord[0][0]))
        
        #use random.choice to select random permuation from list
        feature_unord = np.array([random.choice(feature_perm_lst)])
        
        #append random-ordered array to feature array
        X_features = np.append(X_features,feature_unord,axis = 0)
        
        
    return(X_features)

In [7]:
#get height-wise features
X_features = slice_feature_arrays(df,3)

In [8]:
#reshape X features to 2D array (flatten)
nsamples, nx, ny = X_features.shape
X_features = X_features.reshape((nsamples,nx*ny))

In [9]:
u, inv = np.unique(df.rating, return_inverse=True)
counts = np.bincount(inv)
max_count = max(counts)

# Training ML Classifiers with SMOTE data

## 1a. Base Data

In [10]:
#define features (X) and classification labels (y)
X = X_features
y = np.array(df['rating'][::3])

u, inv = np.unique(y, return_inverse=True)
counts = np.bincount(inv)
max_count_base = max(counts)
print(max_count_base)

226


In [11]:
X.shape, y.shape

((722, 9), (722,))

## 1b. Model 0 Data

In [12]:
bin_lst = []
for i in range(0,df.shape[0]):
    if df['rating'][i] == 1:
        bin_lst.append(1)
    elif df['rating'][i] == 2:
        bin_lst.append(1)
    elif df['rating'][i] == 3:
        bin_lst.append(2)
    elif df['rating'][i] == 4:
        bin_lst.append(3)
    elif df['rating'][i] == 5:
        bin_lst.append(3)

df['severity_level'] = bin_lst

In [13]:
#define features (X) and classification labels (y)
X_m0 = X
y_m0 = np.array(df['severity_level'][::3])

u, inv = np.unique(y_m0, return_inverse=True)
counts = np.bincount(inv)
max_count_m0 = max(counts)
print(max_count_m0)

390


In [14]:
X_m0.shape, y_m0.shape

((722, 9), (722,))

In [15]:
print(counts)

[209 123 390]


## 1c. Model 1 Data

In [16]:
#model1 dataset
X_m1 = X[((y ==1)|(y == 2))]
y_m1 = y[((y == 1)|(y == 2))]

u, inv = np.unique(y_m1, return_inverse=True)
counts = np.bincount(inv)
max_count_m1 = max(counts)
print(max_count_m1)

124


In [17]:
X_m1.shape, y_m1.shape

((209, 9), (209,))

In [18]:
print(counts)

[ 85 124]


## 1d. Model 2 Data

In [19]:
#model2 dataset
X_m2 = X[((y == 4)|(y == 5))]
y_m2 = y[((y == 4)|(y == 5))]

u, inv = np.unique(y_m2, return_inverse=True)
counts = np.bincount(inv)
max_count_m2 = max(counts)
print(counts)

[164 226]


In [20]:
X_m2.shape, y_m2.shape

((390, 9), (390,))

In [21]:
def misclassification_cost(arg1, arg2):
    
    # function for calculating the missclassification cost of a classifier given test labels and predicted labels returned from the
    # trained classifier.
    
    '''
    inputs:
    arg1 = array of test labels
    arg2 = array of predicted labels
    
    returns:
    misclassification cost
    '''

    #print confusion matrix
    CM = confusion_matrix(arg1,arg2)
    print(CM)

    #define cost matrix shape
    cM = np.zeros(CM.shape)

    #assign weights to cost matrix
    if cM.shape == (3,3):
        cM[0] = [0,1,2]
        cM[1] = [1,0,1]
        cM[2] = [2,1,0]

    # for binary classification
    elif cM.shape ==(2,2):
        cM[0] = [0,1]
        cM[1] = [1,0]

    elif cM.shape == (5,5):
        cM[0] = [0,1,2,3,4]
        cM[1] = [1,0,1,2,3]
        cM[2] = [2,1,0,1,2]
        cM[3] = [3,2,1,0,1]
        cM[4] = [4,3,2,1,0]

    #calculate classification cost
    cM_matrix = np.matrix(CM * cM)
    clcost = cM_matrix.sum()/arg2.shape[0]
    
    return(clcost)

In [22]:
#Calculate performance of hierarchical classifier for unbinned as-is data

def add_hierarchical_results(arg1, arg2, arg3):
    
    '''
    This is a function that adds classification performance of the 
    hierarchical classifier to either the unbinned or binned classifier data dataframe
    
    arg1: array of test labels
    arg2: array of predicted labels
    arg3: dataframe returned from classification_pipeline0 for hierarchical classifier data to be added to
    '''
    
    value = 'hierarchical'
    if value in list(arg3['Model']):
        print('hierarchical data already exists')
        return(arg3)
    else:       
  

        report_dict = classification_report(arg1,arg2,output_dict=True)
        model = 'hierarchical'
        accuracy = report_dict['accuracy']
        mpca = report_dict['macro avg']['recall']
        f1_wt = report_dict['weighted avg']['f1-score']
        cost = misclassification_cost(arg1,arg2)
        unique_predictions = np.unique(arg2)

        #Add hierarchical classification results for as-is data to unbinned classification results for as-is data
        table = arg3
        table.loc[len(table.index)] = model,accuracy,mpca,f1_wt,cost,unique_predictions

        return (table)
    

## Define Classification Model

In [60]:
def classification_model(arg1, arg2, arg3, arg4, arg5, arg6, arg7, arg8):
    
    '''
    arg1: Input array (features) - Training data (features)
    arg2: Target array  - Training data (labels)
    arg3: number of items in the majority class (max count)
    arg4: random seed for splitting data
    arg5: oversample rate
    arg6: undersample rate
    arg7: undersample dictionary
    arg8: oversample dictionary
    '''
    
    
    
    SMOTE_dict = {#'SMOTE': 'sampling_strategy = over_sampling_dict,random_state = %d' % (42),
                  #'BorderlineSMOTE': 'sampling_strategy = over_sampling_dict,random_state = %d' % (42),        
                 'SVMSMOTE': 'sampling_strategy = over_sampling_dict,random_state = %d' % (42)}
                 
    
    
    
    
    
    
    #create dataframe for saving results
    output = pd.DataFrame()
    
    
    for key_1 in SMOTE_dict.keys():
        
        print(key_1)
    
    
        #Define classifiers to use during training
        modelDict = {#'DecisionTreeClassifier':'max_depth = %d,random_state = %d' % (4,0),
                     'RandomForestClassifier':'',
                     #'svm.SVC':'kernel="rbf"', 
                     #'KNeighborsClassifier':'n_neighbors = 4',
                     #'LinearDiscriminantAnalysis':'solver = "lsqr",shrinkage = 0.02',
                     #'QuadraticDiscriminantAnalysis':'',
                     #'svm_linear.SVC':'kernel="linear"'
                    }
    
        model_lst = list(modelDict.keys())  


        #create lists for storing model scores for cross-validation
        models = []
        accuracy = []
        mpca = []
        f1_wt = []
        smote = []

        #create lists for storing model scores
        accuracy2 = []
        mpca2 = []
        f1_wt2 = []
        cost2 = []
        unique2 = []
    
    
    
        for key in modelDict.keys():
            print(key)
            models.append(key)
            smote.append(key_1)

            #define classifier with parameters including penalizing parameters
            clf = eval('%s(%s)' % (key,modelDict[key]))

            #define over and under sampling rates
            over_sample_rate = arg5
            under_sample_rate = arg6


            #assign sampling rates to dictionaries
            under_sampling_dict = arg7
            over_sampling_dict = arg8

            # define pipeline
            over = eval('%s(%s)' % (key_1,SMOTE_dict[key_1]))
            under = RandomUnderSampler(sampling_strategy=under_sampling_dict, random_state=1)
            steps = [('o', over), ('u', under), ('model',clf)]
            pipeline = Pipeline(steps=steps)


            #evaluate pipeline

            # define cross-validation method for model evaluation
            cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)       

            #calculate cross-validated mean per class accuracy (recall macroaverage from classification report)
            results = cross_validate(pipeline, arg1, arg2, scoring=['recall_macro','accuracy','f1_macro','f1_weighted'], cv=cv, n_jobs=-1,error_score='raise')

            #append cross-validation results to respective lists
            accuracy.append(results['test_accuracy'].mean())
            mpca.append(results['test_recall_macro'].mean())
            f1_wt.append(results['test_f1_weighted'].mean())





            #train each model with training data and predict on test data

            # Split the data into training and testing sets
            X_train, X_test, y_train, y_test = tts(arg1, arg2, test_size=0.03, random_state=arg4,stratify=arg2)

            # Create a dictionary to store the indices of each label in the training set
            label_indices = {}
            for i in range(len(y_train)):
                label = y_train[i]
                if label not in label_indices:
                    label_indices[label] = []
                label_indices[label].append(i)

            # Create a list to store the 5 samples from each label
            samples = []
            for label in label_indices:
                label_samples = label_indices[label][:5] # Get the first 5 indices for each label
                for idx in label_samples:
                    samples.append(idx)

            # Get the data and labels for the selected samples
            X_test = X_train[samples]
            y_test = y_train[samples]


            #train models
            model = pipeline.fit(X_train, y_train)       

            #predict on test data
            y_hat = model.predict(X_test)      
            print(classification_report(y_test, y_hat))

            #print confusion matrix
            CM = confusion_matrix(y_test,y_hat)
            print(CM)


            #capture classification accuracy metrics
            report_dict = classification_report(y_test,y_hat,output_dict=True)

            #mean per class accuracy
            mpca2.append(report_dict['macro avg']['recall']) #mean per class accuracy

            #return f1 score
            f1_wt2.append(report_dict['weighted avg']['f1-score'])

            #accuracy
            accuracy2.append(report_dict['accuracy'])   

            #misclassification cost
            cost2.append(misclassification_cost(y_test,y_hat))

            #unique label predictions
            unique2.append(np.unique(y_hat))

        
        
        #create and populate dataframe with cross-validation results
        df_scores = pd.DataFrame()
        df_scores['Model'] = models
        df_scores['CV Accuracy'] = accuracy
        df_scores['CV MPCA'] = mpca
        df_scores['CV F1_weighted'] = f1_wt


        #create and populate dataframe with trained model results
        df_scores2 = pd.DataFrame()
        df_scores2['SMOTE MDL'] = smote
        df_scores2['Model'] = models
        df_scores2['Accuracy'] = accuracy2
        df_scores2['MPCA'] = mpca2
        df_scores2['F1_weighted'] = f1_wt2
        df_scores2['Misclassification_Cost'] = cost2
        df_scores2['Unique Predictions'] = unique2
        
        #append results to dataframe
        output = output.append(df_scores2)
        
        print(key_1)
        
        #save dataframe as png
        #df_styled = df_scores2.style.background_gradient()
        #dfi.export(df_styled,'fullcanopy_mb_%s_EXB.png' % (key_1), table_conversion = 'matplotlib')
    
    return(df_scores2,model,X_test,y_test,X_train,y_train,df_scores,output)

In [61]:
#Define SMOTE undersampling and oversampling rates for each model's dataset
'''
osr = oversample rate
usr = undersample rate
usd = undersample dictionary
osd = oversample dictionary
'''

mb_osr = int(0.7*max_count_base)
mb_usr = int(0.7*max_count_base)
mb_usd = {}
mb_osd = {1:mb_osr,2:mb_osr}

m0_osr = int(0.7*max_count_m0)
m0_usr = int(0.7*max_count_m0)
m0_usd = {}
m0_osd = {1:int(0.7*max_count_m0)}

m1_osr = int(0.85*max_count_m1)
m1_usr = int(0.7*max_count_m1)
m1_usd = {}
m1_osd = {1:m1_osr}

m2_osr = int(0.85*max_count_m2)
m2_usr = {}
m2_usd = {}
m2_osd = {}

# Varying random seed when splitting test and training data to assess variance in classifier accuracy

In [62]:
i = 0

# Varying random seed when splitting test and training data to assess variance in classifier accuracy
rdm_seed_lst = [42,43,44,45,46]

#define arguments for the classification model for each dataset
mb_args = [X,y,max_count_base,rdm_seed_lst[i],mb_osr,mb_usr,mb_usd,mb_osd]
m0_args = [X_m0,y_m0,max_count_m0,rdm_seed_lst[i],m0_osr,m0_usr,m0_usd,m0_osd]
m1_args = [X_m1,y_m1,max_count_m1,rdm_seed_lst[i],m1_osr,m1_usr,m1_usd,m1_osd]
m2_args = [X_m2,y_m2,max_count_m2,rdm_seed_lst[i],m2_osr,m2_usr,m2_usd,m2_osd]

# train model on each dataset using the arguments defined above

output_mb = pd.DataFrame()
output_m0 = pd.DataFrame()
output_m1 = pd.DataFrame()
output_m2 = pd.DataFrame()


for i in range(len(rdm_seed_lst)): 
    mb_results = classification_model(*mb_args)
    m0_results = classification_model(*m0_args)
    m1_results = classification_model(*m1_args)
    m2_results = classification_model(*m2_args)
    
    output_mb = output_mb.append(mb_results[7])
    output_m0 = output_m0.append(m0_results[7])
    output_m1 = output_m1.append(m1_results[7])
    output_m2 = output_m2.append(m2_results[7])
    
#output.to_excel('mb_output_2021colors_EX1B.xlsx')

SVMSMOTE
RandomForestClassifier
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         5
           2       1.00      1.00      1.00         5
           3       1.00      1.00      1.00         5
           4       1.00      1.00      1.00         5
           5       1.00      1.00      1.00         5

    accuracy                           1.00        25
   macro avg       1.00      1.00      1.00        25
weighted avg       1.00      1.00      1.00        25

[[5 0 0 0 0]
 [0 5 0 0 0]
 [0 0 5 0 0]
 [0 0 0 5 0]
 [0 0 0 0 5]]
[[5 0 0 0 0]
 [0 5 0 0 0]
 [0 0 5 0 0]
 [0 0 0 5 0]
 [0 0 0 0 5]]
SVMSMOTE
SVMSMOTE
RandomForestClassifier
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         5
           2       1.00      1.00      1.00         5
           3       1.00      1.00      1.00         5

    accuracy                           1.00        15
   macro avg       1.00      1.00 

              precision    recall  f1-score   support

           1       1.00      1.00      1.00         5
           2       1.00      1.00      1.00         5
           3       1.00      1.00      1.00         5

    accuracy                           1.00        15
   macro avg       1.00      1.00      1.00        15
weighted avg       1.00      1.00      1.00        15

[[5 0 0]
 [0 5 0]
 [0 0 5]]
[[5 0 0]
 [0 5 0]
 [0 0 5]]
SVMSMOTE
SVMSMOTE
RandomForestClassifier
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         5
           2       1.00      1.00      1.00         5

    accuracy                           1.00        10
   macro avg       1.00      1.00      1.00        10
weighted avg       1.00      1.00      1.00        10

[[5 0]
 [0 5]]
[[5 0]
 [0 5]]
SVMSMOTE
SVMSMOTE
RandomForestClassifier
              precision    recall  f1-score   support

           4       1.00      1.00      1.00         5
           5   

# Hierarchical classification with trained models (M0, M1, and M2)

### <u>Logic</u>
1. Use Model M0 to predict the bin (1, 2, or 3) the sample belongs to.
2. If the predicted bin is 1, use Model M1 to predict an IDC rating of 1 or 2.
3. Else if Model M0 predicts the bin to be 2, return an IDC Rating of 3.
4. Else if Model M0 predicts the bin to be 3, use Model M2 to predict and IDC Rating of 4 or 5.

In [63]:
#hierarchical classification of validation data

rdm_seed_lst = [42,43,44,45,46]


seed = []
mpca = []
f1_wt = []
cost = []
accuracy = []
unique_predictions = []



for i in range(len(rdm_seed_lst)):
    seed.append(rdm_seed_lst[i])
    
    #mb = smote_classifier_base(X,y,max_count_base,rdm_seed_lst[i
    X_train, X_test, y_train, y_test = tts(X,y, random_state=i, stratify = y)
    mbx = [X_train,  y_train, X_test, y_test]

    pred_list = []
    for i in range(0,len(mbx[2])):
        if m0_results[1].predict(mbx[2])[i] == 1:    #idc score is 1 or 2
            pred = m1_results[1].predict(mbx[2])[i]  #use model 1 to classify as 1 or 2
            pred_list.append(pred)                          
        elif m0_results[1].predict(mbx[2])[i] == 2:
            pred = 3
            pred_list.append(pred)
        elif m0_results[1].predict(mbx[2])[i] == 3:   #idc score is 4 or 5
            pred = m2_results[1].predict(mbx[2])[i]   #use model 2 to classify as 4 or 5
            pred_list.append(pred)
    pred_list = np.array(pred_list)                         

    print(classification_report(mbx[3],pred_list))
    
    #print classification report
    report_dict = classification_report(mbx[3],pred_list,output_dict=True)

    #mean per-class accuracy
    mpca.append(report_dict['macro avg']['recall']) #mean per class accuracy

    #return f1 score
    f1_wt.append(report_dict['weighted avg']['f1-score'])

    #accuracy
    accuracy.append(accuracy_score(pred_list,mbx[3]))       

    #uniqueness of the predictions
    unique_predictions.append(np.unique(pred_list))

    #misclassification cost
    cost.append(misclassification_cost(mbx[3],pred_list))
    
df_hierarchy_cv = pd.DataFrame()
df_hierarchy_cv['Seed'] = seed
df_hierarchy_cv['Accuracy'] = accuracy
df_hierarchy_cv['MPCA'] = mpca
df_hierarchy_cv['F1_weighted'] = f1_wt
df_hierarchy_cv['Misclassification Cost'] = cost

#df_hierarchy_cv.to_excel('hierarchy_results.xlsx')

              precision    recall  f1-score   support

           1       1.00      1.00      1.00        21
           2       1.00      1.00      1.00        31
           3       1.00      0.97      0.98        31
           4       0.98      0.98      0.98        41
           5       0.98      1.00      0.99        57

    accuracy                           0.99       181
   macro avg       0.99      0.99      0.99       181
weighted avg       0.99      0.99      0.99       181

[[21  0  0  0  0]
 [ 0 31  0  0  0]
 [ 0  0 30  1  0]
 [ 0  0  0 40  1]
 [ 0  0  0  0 57]]
              precision    recall  f1-score   support

           1       1.00      1.00      1.00        21
           2       1.00      1.00      1.00        31
           3       1.00      0.97      0.98        31
           4       0.98      0.98      0.98        41
           5       0.98      1.00      0.99        57

    accuracy                           0.99       181
   macro avg       0.99      0.99      0

In [64]:
df_hierarchy_cv

,Seed,Accuracy,MPCA,F1_weighted,Misclassification Cost
0,42,0.98895,0.98867,0.988929,0.01105
1,43,0.98895,0.98867,0.988929,0.01105
2,44,1.00000,1.00000,1.000000,0.00000
3,45,1.00000,1.00000,1.000000,0.00000
4,46,1.00000,1.00000,1.000000,0.00000


In [59]:
df_hierarchy_cv

,Seed,Accuracy,MPCA,F1_weighted,Misclassification Cost
0,42,0.928177,0.929883,0.927984,0.071823
1,43,0.906077,0.904280,0.905217,0.093923
2,44,0.939227,0.935327,0.939076,0.060773
3,45,0.895028,0.895894,0.891895,0.110497
4,46,0.917127,0.914036,0.915984,0.082873
